In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from networkx.algorithms import bipartite 
import csv
import itertools
%matplotlib inline


In [2]:
db_data = pd.read_csv("uniprotlinks.csv")
unip_data = pd.read_csv("uniprot-all2.csv")

In [3]:
df = db_data.rename(columns={'Name': 'Drug Name', 'UniProt Name': 'Protein Name'})
df_2 = unip_data.rename(columns={'Entry': 'UniProt ID', 'Protein names': 'Protein Name', 'Gene names': 'Gene Symbol'})

In [4]:
##Chaning the type of column "Gene Symbol" to list. 
Gene_sym = df_2["Gene Symbol"].tolist()

##Splitting data into list of lists using space delimiter. 
what_i_want = []
for id_ in Gene_sym:
    temp = str(id_).split(' ')
    what_i_want.append(temp)
    
#Trimming off Gene Symbol Aliases 

what_i_actually_want = [id_pair[0] for id_pair in what_i_want]  ##list comprehensions 
    

In [5]:
s = pd.Series(what_i_actually_want)  ##turning the list of values into a series.
df_2['Gene Symbol'] = s.values   ##adding that series to the dataframe. 

In [6]:
##Using merge to merge two dataframes and map the UniProt Id to Gene Symbol. 

combined_df = db_data.merge(df_2, how="left")

In [7]:
##rename 'Name' column in combined_df to 'Drug Name' and 'Type' column to 'Drug Type'.

combined_df2 = combined_df.rename(columns={'Name': 'Drug Name', 'Type': 'Drug Type', 'UniProt Name': 'Protein Names'})
del combined_df2["Protein Name"]

In [8]:
gene_sym = combined_df2["Gene Symbol"]
gene_sym_isnull = pd.isnull(gene_sym)
gene_sym_nulltrue = gene_sym[gene_sym_isnull]
gene_sym_nullfalse = gene_sym[gene_sym_isnull == False]

In [9]:
print(len(gene_sym_nulltrue))

4140


In [10]:
gene_sym_nullfalse.shape

(12028,)

In [11]:
##Dropping rows with empty columns in Gene Symbol. 

combined_df2.dropna(subset=['Gene Symbol'], inplace=True)
#combined_df2.to_csv('drug_gene.csv', sep='\t')

In [12]:
combined_df2.shape

(12028, 7)

In [13]:
B = nx.Graph()

B.add_nodes_from(combined_df2['Drug Name'], node_type = 'D')

B.add_nodes_from(combined_df2['Gene Symbol'], node_type = 'Genes')

B.add_edges_from([(row['Drug Name'], row['Gene Symbol']) for idx, row in combined_df2.iterrows()], edge_type = 'D_targ')

nx.info(B)

'Name: \nType: Graph\nNumber of nodes: 7191\nNumber of edges: 11825\nAverage degree:   3.2888'

In [14]:
#remove_edges = [('Tamoxifen','EBP'), ('Tamoxifen', 'PRKCA'), ('Tamoxifen', 'PRKCB'), ('Tamoxifen', 'PRKCD'), ('Tamoxifen', 'PRKCE'), ('Tamoxifen', 'PRKCG'), ('Tamoxifen', 'PRKCI'), ('Tamoxifen', 'PRKCQ'), ('Tamoxifen', 'PRKCZ')]

In [15]:
#B.remove_edges_from(remove_edges)

In [16]:
#separating the two sets of nodes: drugs and genes. 

d, g = bipartite.sets(B)


In [17]:
#degree distrubtion of drug nodes. 
d_degree = B.degree(d)

#degree distribution of gene nodes. 
g_degree = B.degree(g)

In [18]:
string = pd.read_csv("HumanStringNet.txt", delimiter="\t", header=None)

In [19]:
string_df = string.rename(columns={0: 'Gene Name', 1: 'Interacting Gene', 2: 'Interaction Weight'})

In [20]:
string_df['Gene_Gene'] = string_df[['Gene Name', 'Interacting Gene']].apply(tuple, axis=1)

In [21]:
edges = string_df["Gene_Gene"].tolist()
G = nx.Graph()
G.add_edges_from(edges, edge_type='gene_gene')
degree = G.degree()

nx.info(G)

'Name: \nType: Graph\nNumber of nodes: 9215\nNumber of edges: 160249\nAverage degree:  34.7800'

In [22]:
max(degree, key=degree.get)  
degree['UBC']

1746

In [23]:
"""Takes the drug as node argument to find the genes that a specific drug works on in graph B(drug-gene graph),
   then, finding the neighboring genes of those genes in G(gene-gene graph).
   Results: dictionary key-value pairs."""

def nbr_nodes(node):
    graph = nx.Graph()
    drug_to_gene_nodes = B.neighbors(node)
    neighbors = {}
    
    for dg_node in drug_to_gene_nodes:
        neighbors[dg_node] = {}
        neighbors[dg_node] = []
        
        
        if G.has_node(dg_node):
            gg_nodes = G.neighbors(dg_node)
            
            for ggn in gg_nodes:
                if G.has_node(ggn):
                    neighbors[dg_node].append(ggn)
                
                    
    return(neighbors)

In [24]:
drug_mutation = pd.read_csv("drug_mutation1.csv")

In [25]:
drug_mutation.shape

(254881, 7)

In [26]:
del drug_mutation['Unnamed: 0']

In [27]:
drug_mutation['Drug_list'] = [x.strip().replace(' Methotrexate', 'Methoxtrexate') for x in drug_mutation['Drug_list']]
drug_mutation['Drug_list'] = [x.strip().replace(' Fluorouracil', 'Fluorouracil') for x in drug_mutation['Drug_list']]
drug_mutation['Drug_list'] = [x.strip().replace(' Cyclophosphamide', 'Cyclophosphamide') for x in drug_mutation['Drug_list']]
drug_mutation['Drug_list'] = [x.strip().replace(' Bevacizumab', 'Bevacizumab') for x in drug_mutation['Drug_list']]
drug_mutation['Drug_list'] = [x.strip().replace(' Tamoxifen', 'Tamoxifen') for x in drug_mutation['Drug_list']]
drug_mutation['Drug_list'] = [x.strip().replace(' Aromatase Inhibitor', 'Aromatase Inhibitor') for x in drug_mutation['Drug_list']]
drug_mutation['Drug_list'] = [x.strip().replace('Doxorubincin', 'Doxorubicin') for x in drug_mutation['Drug_list']]
drug_mutation['Drug_list'] = [x.strip().replace('Methoxtrexate', 'Methotrexate') for x in drug_mutation['Drug_list']]


In [28]:
gene_affected = drug_mutation["Gene name"]
gene_affected_isnull = pd.isnull(gene_affected)
gene_affected_nulltrue = gene_affected[gene_affected_isnull]
gene_affected_nullfalse = gene_affected[gene_affected_isnull == False]
drug_mutation.dropna(subset=['Gene name'], inplace=True)

In [29]:
drug_list = drug_mutation['Drug_list'].drop_duplicates()

In [30]:
def target_paths(node):
    
    drug_to_gene_nodes = B.neighbors(node)
    neighbors = {}
    for dg_node in drug_to_gene_nodes:
        neighbors[dg_node] = {}
        
        if G.has_node(dg_node):
            
            path = nx.single_source_shortest_path(G, dg_node, 1)
            nbr_path = (path)
            neighbors.update(nbr_path)
            
    return(neighbors)

In [31]:
def target_paths2(node):
    
    drug_to_gene_nodes = B.neighbors(node)
    neighbors = {}
    for dg_node in drug_to_gene_nodes:
        neighbors[dg_node] = {}
        
        if G.has_node(dg_node):
            
            path = nx.single_source_shortest_path(G, dg_node, 2)
            nbr_path = (path)
            neighbors.update(nbr_path)
            
    return(neighbors)

In [32]:
def tidy_split(df, column, sep='|', keep=False):
    """
    Split the values of a column and expand so the new DataFrame has one split
    value per row. Filters rows where the column is missing.

    Params
    ------
    df : pandas.DataFrame
        dataframe with the column to split and expand
    column : str
        the column to split and expand
    sep : str
        the string used to split the column's values
    keep : bool
        whether to retain the presplit value as it's own row

    Returns
    -------
    pandas.DataFrame
        Returns a dataframe with the same columns as `df`.
    """
    indexes = list()
    new_values = list()
    df = df.dropna(subset=[column])
    for i, presplit in enumerate(df[column].astype(str)):
        values = presplit.split(sep)
        if keep and len(values) > 1:
            indexes.append(i)
            new_values.append(presplit)
        for value in values:
            indexes.append(i)
            new_values.append(value)
    new_df = df.iloc[indexes, :].copy()
    new_df[column] = new_values
    return new_df

In [33]:
drug_mutation1 = tidy_split(drug_mutation, "Drug_list", sep=',' )
drug_mutation1.shape

(465377, 6)

In [34]:
mu_pi_dtarg = pd.DataFrame()
for pi in set(drug_mutation1["icgc_donor_id"]):
    mu_pi_dtarg[pi] = pd.Series('P')

    for drug in set(drug_mutation1['Drug_list']):
        mu_pi_dtarg[drug] = pd.Series('D')


In [35]:
mu_pi_dtarg = mu_pi_dtarg.T

In [36]:
mu_pi_dtarg.to_csv('pi_drug.noa', sep='\t')

In [37]:
Anastrozole = target_paths("Anastrozole")
Fulvestrant = target_paths("Fulvestrant")
Capecitabine = target_paths("Capecitabine")
Vinorelbine  = target_paths("Vinorelbine")
Docetaxel  = target_paths("Docetaxel")
Letrozole = target_paths("Letrozole")
Methotrexate  = target_paths("Methotrexate")
Fluorouracil  = target_paths("Fluorouracil")
Dexrazoxane  = target_paths("Dexrazoxane")
Doxorubicin  = target_paths("Doxorubicin")
Tamoxifen  = target_paths("Tamoxifen")
Paclitaxel  = target_paths("Paclitaxel")
Bevacizumab = target_paths("Bevacizumab")
Trastuzumab = target_paths("Trastuzumab")
Exemestane = target_paths("Exemestane")
Epirubicin  = target_paths("Epirubicin")
Gemcitabine  = target_paths("Gemcitabine")
Alendronic_acid  = target_paths("Alendronic acid")
Buserelin = target_paths("Buserelin")
Zoledronic_acid = target_paths("Zoledronic acid")

In [38]:
tam_graph = nx.from_dict_of_lists(Tamoxifen,create_using=None) 

In [39]:
nx.info(tam_graph)

'Name: \nType: Graph\nNumber of nodes: 439\nNumber of edges: 872\nAverage degree:   3.9727'

In [40]:
G_tam = G.subgraph(tam_graph.nodes())

In [41]:
nx.info(G_tam)

'Name: \nType: Graph\nNumber of nodes: 438\nNumber of edges: 6259\nAverage degree:  28.5799'

In [42]:
pi_tam_mutated = drug_mutation1[drug_mutation1['Drug_list'] == 'Tamoxifen']

In [43]:
pi_tam_mutated.shape

(116730, 6)

In [44]:
G_tam = G.subgraph(tam_graph.nodes())

# add edges between drug and drug target 
#for tam, there's only two targets after removing unneccesary targets (ESR1, ESR2 (both are antagonist, agonist))
G_tam.add_edges_from([("Tamoxifen", x) for x in B.neighbors("Tamoxifen")], edge_type = 'tam_targ')

# mutated genes in the subnetwork
common_genes = set(G_tam.nodes()).intersection(pi_tam_mutated["Gene name"])
common_pi_tam_mutated = pi_tam_mutated[pi_tam_mutated["Gene name"].isin(common_genes)]
# add edges between patients and mutated genes 
mutated_pi = common_pi_tam_mutated["icgc_donor_id"].tolist()
mutated_gene = common_pi_tam_mutated["Gene name"].tolist()
G_tam.add_edges_from(zip(mutated_pi, mutated_gene), edge_type = 'pi_tam_mu')

#nx.write_graphml(G_tam, 'Tam_pi_mu_all.graphml')

In [45]:
nx.info(G_tam)

'Name: \nType: Graph\nNumber of nodes: 491\nNumber of edges: 8551\nAverage degree:  34.8310'

In [46]:
len(set(mutated_pi))

51

In [47]:
len(set(mutated_gene))

391

In [48]:
tam_mu_genes = pd.DataFrame()
for genes in pi_tam_mutated['Gene name']:
    tam_mu_genes[genes] = pd.Series('tam_mu_genes')

tam_mu_genes = tam_mu_genes.T

tam_mu_genes.to_csv("tam_mu_genes.noa", sep='\t')

In [49]:
pi_anas_mutated = drug_mutation1[drug_mutation1['Drug_list'] == 'Anastrozole']

In [50]:
len(set(pi_anas_mutated['icgc_donor_id']))

14

In [51]:
Anastrozole_g = nx.from_dict_of_lists(Anastrozole,create_using=None)

G_anastrozole = G.subgraph(Anastrozole_g.nodes())

# add edges between drug and drug target 
G_anastrozole.add_edges_from([("Anastrozole", x) for x in B.neighbors("Anastrozole")], edge_type = 'anas_targ')

# mutated genes in the subnetwork
common_genes2 = set(G_anastrozole.nodes()).intersection(pi_anas_mutated["Gene name"])
common_pi_anas_mutated = pi_anas_mutated[pi_anas_mutated["Gene name"].isin(common_genes2)]

# add edges between patients and mutated genes 
mutated_pi2 = common_pi_anas_mutated["icgc_donor_id"].tolist()
mutated_gene2 = common_pi_anas_mutated["Gene name"].tolist()
G_anastrozole.add_edges_from(zip(mutated_pi2, mutated_gene2), edge_type = 'pi_mutation_anas')


nx.write_graphml(G_anastrozole, 'Anas_pi_mu_all.graphml')

nx.info(G_anastrozole)

'Name: \nType: Graph\nNumber of nodes: 43\nNumber of edges: 308\nAverage degree:  14.3256'

In [52]:
Anastrozole_g = nx.from_dict_of_lists(Anastrozole,create_using=None)

G_anastrozole = G.subgraph(Anastrozole_g.nodes())

# add edges between drug and drug target 
G_anastrozole.add_edges_from([("Anastrozole", x) for x in B.neighbors("Anastrozole")], edge_type = 'anas_targ')

In [53]:
df = list(pi_anas_mutated['Gene name'])

In [54]:
len(mutated_gene2)

24

In [55]:
mutated_anas = pi_anas_mutated['Gene name']

In [56]:
anas_mu_genes = pd.DataFrame()
for genes in pi_anas_mutated['Gene name']:
    anas_mu_genes[genes] = pd.Series('anas_mu_genes')

anas_mu_genes = anas_mu_genes.T

anas_mu_genes.to_csv("anas_mu_genes.noa", sep='\t')

In [57]:
pi_dex_mutated = drug_mutation1[drug_mutation1['Drug_list'] == 'Dexrazoxane']

Dexrazoxane_g = nx.from_dict_of_lists(Dexrazoxane,create_using=None) 

G_dexrazoxane = G.subgraph(Dexrazoxane_g.nodes())

# add edges between drug and drug target 
G_dexrazoxane.add_edges_from([("Dexrazoxane", x) for x in B.neighbors("Dexrazoxane")], edge_type = 'drug_targ')

# mutated genes in the subnetwork
common_genes3 = set(G_dexrazoxane.nodes()).intersection(pi_dex_mutated["Gene name"])
common_pi_dex_mutated = pi_dex_mutated[pi_dex_mutated["Gene name"].isin(common_genes3)]

# add edges between patients and mutated genes 
mutated_pi3 = common_pi_dex_mutated["icgc_donor_id"].tolist()
mutated_gene3 = common_pi_dex_mutated["Gene name"].tolist()
G_dexrazoxane.add_edges_from(zip(mutated_pi3, mutated_gene3), edge_type = 'pi_dex_mu')

nx.write_graphml(G_dexrazoxane, 'Dex_pi_mu_all.graphml')

nx.info(G_dexrazoxane)

'Name: \nType: Graph\nNumber of nodes: 53\nNumber of edges: 284\nAverage degree:  10.7170'

In [58]:
dex_mu_genes = pd.DataFrame()
for genes in pi_dex_mutated['Gene name']:
    dex_mu_genes[genes] = pd.Series('dex_mu_genes')

dex_mu_genes =dex_mu_genes.T

dex_mu_genes.to_csv("dex_mu_genes.noa", sep='\t')

In [59]:
pi_exem_mutated = drug_mutation1[drug_mutation1['Drug_list'] == 'Exemestane']

Exemestane_g = nx.from_dict_of_lists(Exemestane,create_using=None) 

G_Exemestane = G.subgraph(Exemestane_g.nodes())

# add edges between drug and drug target 
G_Exemestane.add_edges_from([("Exemestane", x) for x in B.neighbors("Exemestane")], edge_type = 'drug_targ')

# mutated genes in the subnetwork
common_genes4 = set(G_Exemestane.nodes()).intersection(pi_exem_mutated["Gene name"])
common_pi_exem_mutated = pi_exem_mutated[pi_exem_mutated["Gene name"].isin(common_genes4)]

# add edges between patients and mutated genes 
mutated_pi4 = common_pi_exem_mutated["icgc_donor_id"].tolist()
mutated_gene4 = common_pi_exem_mutated["Gene name"].tolist()
Exemestane_g.add_edges_from(zip(mutated_pi4, mutated_gene4), edge_type ='pi_exem_mu')

nx.write_graphml(G_Exemestane, 'Exem_pi_mu_all.graphml')
##2-hop, 346 nodes, 4108 edges 
nx.info(G_Exemestane)

'Name: \nType: Graph\nNumber of nodes: 34\nNumber of edges: 284\nAverage degree:  16.7059'

In [60]:
pi_letro_mutated = drug_mutation1[drug_mutation1['Drug_list'] == 'Letrozole']

In [61]:
pi_letro_mutated.shape

(14267, 6)

In [62]:
letro = target_paths2("Letrozole")

In [63]:
Letrozole_g = nx.from_dict_of_lists(letro,create_using=None) 

G_Letrozole = G.subgraph(Letrozole_g.nodes())

# add edges between drug and drug target 
G_Letrozole.add_edges_from([("Letrozole", x) for x in B.neighbors("Letrozole")], edge_type = 'drug_targ')

# mutated genes in the subnetwork
common_genes5 = set(G_Letrozole.nodes()).intersection(pi_letro_mutated["Gene name"])
common_pi_letro_mutated = pi_letro_mutated[pi_letro_mutated["Gene name"].isin(common_genes5)]

# add edges between patients and mutated genes 
mutated_pi5 = common_pi_letro_mutated["icgc_donor_id"].tolist()
mutated_gene5 = common_pi_letro_mutated["Gene name"].tolist()
G_Letrozole.add_edges_from(zip(mutated_pi5, mutated_gene5), edge_type = 'pi_letro_mu')

nx.write_graphml(G_Letrozole, 'Letro_pi_mu_all.graphml')

nx.info(G_Letrozole)

'Name: \nType: Graph\nNumber of nodes: 350\nNumber of edges: 4313\nAverage degree:  24.6457'

In [64]:
letro_mu_genes = pd.DataFrame()
for genes in pi_letro_mutated['Gene name']:
    letro_mu_genes[genes] = pd.Series('letro_mu_genes')

letro_mu_genes =letro_mu_genes.T
letro_mu_genes.to_csv("letro_mu_genes.noa", sep='\t')

In [65]:
pi_doce_mutated = drug_mutation1[drug_mutation1['Drug_list'] == 'Docetaxel']

Docetaxel_g = nx.from_dict_of_lists(Docetaxel,create_using=None)

G_Docetaxel = G.subgraph(Docetaxel_g.nodes())

# add edges between drug and drug target 
G_Docetaxel.add_edges_from([("Docetaxel", x) for x in B.neighbors("Docetaxel")], edge_type ='drug_targ')

# mutated genes in the subnetwork
common_genes6 = set(G_Docetaxel.nodes()).intersection(pi_doce_mutated["Gene name"])
common_pi_doce_mutated = pi_doce_mutated[pi_doce_mutated["Gene name"].isin(common_genes6)]

# add edges between patients and mutated genes 

mutated_pi6 = common_pi_doce_mutated["icgc_donor_id"].tolist()
mutated_gene6 = common_pi_doce_mutated["Gene name"].tolist()
G_Docetaxel.add_edges_from(zip(mutated_pi6, mutated_gene6), edge_type = 'pi_doce_mu')

nx.write_graphml(G_Docetaxel, 'Doce_pi_mu_all.graphml')

nx.info(G_Docetaxel)

'Name: \nType: Graph\nNumber of nodes: 75\nNumber of edges: 288\nAverage degree:   7.6800'

In [66]:
doce_mu_genes = pd.DataFrame()
for genes in pi_doce_mutated['Gene name']:
    doce_mu_genes[genes] = pd.Series('doce_mu_genes')

doce_mu_genes =doce_mu_genes.T
doce_mu_genes.to_csv("doce_mu_genes.noa", sep='\t')

In [67]:
pi_fulv_mutated = drug_mutation1[drug_mutation1['Drug_list'] == 'Fulvestrant']

Fulvestrant_g = nx.from_dict_of_lists(Fulvestrant,create_using=None)

G_Fulvestrant = G.subgraph(Fulvestrant_g.nodes())

# add edges between drug and drug target 
G_Fulvestrant.add_edges_from([("Fulvestrant", x) for x in B.neighbors("Fulvestrant")], edge_type = 'drug_targ')

# mutated genes in the subnetwork
common_genes7 = set(G_Fulvestrant.nodes()).intersection(pi_fulv_mutated["Gene name"])
common_pi_fulv_mutated = pi_fulv_mutated[pi_fulv_mutated["Gene name"].isin(common_genes7)]

# add edges between patients and mutated genes 

mutated_pi7 = common_pi_fulv_mutated["icgc_donor_id"].tolist()
mutated_gene7 = common_pi_fulv_mutated["Gene name"].tolist()
G_Fulvestrant.add_edges_from(zip(mutated_pi7, mutated_gene7), edge_type = 'pi_fulv_mu')

nx.write_graphml(G_Fulvestrant, 'Fulv_pi_mu_all.graphml')

nx.info(G_Fulvestrant)

'Name: \nType: Graph\nNumber of nodes: 197\nNumber of edges: 2149\nAverage degree:  21.8173'

In [68]:
fulv_mu_genes = pd.DataFrame()
for genes in pi_fulv_mutated['Gene name']:
    fulv_mu_genes[genes] = pd.Series('fulv_mu_genes')

fulv_mu_genes =fulv_mu_genes.T
fulv_mu_genes.to_csv("fulv_mu_genes.noa", sep='\t')

In [69]:
pi_capec_mutated = drug_mutation1[drug_mutation1['Drug_list'] == 'Capecitabine']

Capecitabine_g = nx.from_dict_of_lists(Capecitabine,create_using=None)

G_Capecitabine = G.subgraph(Capecitabine_g.nodes())

# add edges between drug and drug target 
G_Capecitabine.add_edges_from([("Capecitabine", x) for x in B.neighbors("Capecitabine")], edge_type = 'drug_targ')

# mutated genes in the subnetwork
common_genes8 = set(G_Capecitabine.nodes()).intersection(pi_capec_mutated["Gene name"])
common_pi_capec_mutated = pi_capec_mutated[pi_capec_mutated["Gene name"].isin(common_genes8)]

# add edges between patients and mutated genes 

mutated_pi8 = common_pi_capec_mutated["icgc_donor_id"].tolist()
mutated_gene8 = common_pi_capec_mutated["Gene name"].tolist()
G_Capecitabine.add_edges_from(zip(mutated_pi8, mutated_gene8), edge_type = 'pi_capec_mu')

nx.write_graphml(G_Capecitabine, 'Capec_pi_mu_all.graphml')

nx.info(G_Capecitabine)

'Name: \nType: Graph\nNumber of nodes: 41\nNumber of edges: 194\nAverage degree:   9.4634'

In [70]:
capec_mu_genes = pd.DataFrame()
for genes in pi_capec_mutated['Gene name']:
    capec_mu_genes[genes] = pd.Series('capec_mu_genes')

capec_mu_genes =capec_mu_genes.T
capec_mu_genes.to_csv("capec_mu_genes.noa", sep='\t')

In [71]:
pi_vinor_mutated = drug_mutation1[drug_mutation1['Drug_list'] == 'Vinorelbine']

Vinorelbine_g = nx.from_dict_of_lists(Vinorelbine,create_using=None)

G_Vinorelbine = G.subgraph(Vinorelbine_g.nodes())

# add edges between drug and drug target 
G_Vinorelbine.add_edges_from([("Vinorelbine", x) for x in B.neighbors("Vinorelbine")], edge_type = 'drug_targ')

# mutated genes in the subnetwork
common_genes9 = set(G_Vinorelbine.nodes()).intersection(pi_vinor_mutated["Gene name"])
common_pi_vinor_mutated = pi_vinor_mutated[pi_vinor_mutated["Gene name"].isin(common_genes9)]

# add edges between patients and mutated genes 

mutated_pi9 = common_pi_vinor_mutated["icgc_donor_id"].tolist()
mutated_gene9 = common_pi_vinor_mutated["Gene name"].tolist()
G_Vinorelbine.add_edges_from(zip(mutated_pi9, mutated_gene9), edge_type = 'pi_vinor_mu')

nx.write_graphml(G_Vinorelbine, 'Vinor_pi_mu_all.graphml')

nx.info(G_Vinorelbine)

'Name: \nType: Graph\nNumber of nodes: 2\nNumber of edges: 1\nAverage degree:   1.0000'

In [72]:
B.neighbors("Vinorelbine")     #not in string network

['TUBB']

In [73]:
pi_metho_mutated = drug_mutation1[drug_mutation1['Drug_list'] == 'Methotrexate']

Methotrexate_g = nx.from_dict_of_lists(Methotrexate,create_using=None)

G_Methotrexate = G.subgraph(Methotrexate_g.nodes())

# add edges between drug and drug target 
G_Methotrexate.add_edges_from([("Methotrexate", x) for x in B.neighbors("Methotrexate")], edge_type= 'drug_targ')

# mutated genes in the subnetwork
common_genes10 = set(G_Methotrexate.nodes()).intersection(pi_metho_mutated["Gene name"])
common_pi_metho_mutated = pi_metho_mutated[pi_metho_mutated["Gene name"].isin(common_genes10)]

# add edges between patients and mutated genes 

mutated_pi10 = common_pi_metho_mutated["icgc_donor_id"].tolist()
mutated_gene10 = common_pi_metho_mutated["Gene name"].tolist()
G_Methotrexate.add_edges_from(zip(mutated_pi10, mutated_gene10), edge_type='pi_metho_mu')

nx.write_graphml(G_Methotrexate, 'Metho_pi_mu_all.graphml')

nx.info(G_Methotrexate)

'Name: \nType: Graph\nNumber of nodes: 3\nNumber of edges: 2\nAverage degree:   1.3333'

In [74]:
B.neighbors('Methotrexate')       #not in string network

['DHFR']

In [75]:
pi_fluo_mutated = drug_mutation1[drug_mutation1['Drug_list'] == 'Fluorouracil']

Fluorouracil_g = nx.from_dict_of_lists(Fluorouracil,create_using=None)

G_Fluorouracil = G.subgraph(Fluorouracil_g.nodes())

# add edges between drug and drug target 
G_Fluorouracil.add_edges_from([("Fluorouracil", x) for x in B.neighbors("Fluorouracil")], edge_type ='drug_targ')

# mutated genes in the subnetwork
common_genes11 = set(G_Fluorouracil.nodes()).intersection(pi_fluo_mutated["Gene name"])
common_pi_fluo_mutated = pi_fluo_mutated[pi_fluo_mutated["Gene name"].isin(common_genes11)]

# add edges between patients and mutated genes 

mutated_pi11 = common_pi_fluo_mutated["icgc_donor_id"].tolist()
mutated_gene11 = common_pi_fluo_mutated["Gene name"].tolist()
G_Fluorouracil.add_edges_from(zip(mutated_pi11, mutated_gene11), edge_type = 'pi_fluo_mu')

nx.write_graphml(G_Fluorouracil, 'Fluo_pi_mu_all.graphml')

nx.info(G_Fluorouracil)

'Name: \nType: Graph\nNumber of nodes: 53\nNumber of edges: 226\nAverage degree:   8.5283'

In [76]:
fluo_mu_genes = pd.DataFrame()
for genes in pi_fluo_mutated['Gene name']:
    fluo_mu_genes[genes] = pd.Series('fluo_mu_genes')

fluo_mu_genes =fluo_mu_genes.T
fluo_mu_genes.to_csv("fluo_mu_genes.noa", sep='\t')

In [77]:
pi_doxo_mutated = drug_mutation1[drug_mutation1['Drug_list'] == 'Doxorubicin']

Doxorubicin_g = nx.from_dict_of_lists(Doxorubicin,create_using=None)

G_Doxorubicin = G.subgraph(Doxorubicin_g.nodes())

# add edges between drug and drug target 
G_Doxorubicin.add_edges_from([("Doxorubicin", x) for x in B.neighbors("Doxorubicin")], edge_type = 'drug_targ')

# mutated genes in the subnetwork
common_genes12 = set(G_Doxorubicin.nodes()).intersection(pi_doxo_mutated["Gene name"])
common_pi_doxo_mutated = pi_doxo_mutated[pi_doxo_mutated["Gene name"].isin(common_genes12)]

# add edges between patients and mutated genes 

mutated_pi12 = common_pi_fluo_mutated["icgc_donor_id"].tolist()
mutated_gene12 = common_pi_fluo_mutated["Gene name"].tolist()
G_Doxorubicin.add_edges_from(zip(mutated_pi12, mutated_gene12), edge_type = 'pi_doxo_mu')

nx.write_graphml(G_Doxorubicin, 'Doxo_pi_mu_all.graphml')

nx.info(G_Doxorubicin)

'Name: \nType: Graph\nNumber of nodes: 65\nNumber of edges: 213\nAverage degree:   6.5538'

In [78]:
doxo_mu_genes = pd.DataFrame()
for genes in pi_doxo_mutated['Gene name']:
    fulv_mu_genes[genes] = pd.Series('doxo_mu_genes')

doxo_mu_genes =doxo_mu_genes.T
doxo_mu_genes.to_csv("doxo_mu_genes.noa", sep='\t')

In [79]:
pi_taxel_mutated = drug_mutation1[drug_mutation1['Drug_list'] == 'Paclitaxel']

Paclitaxel_g = nx.from_dict_of_lists(Paclitaxel,create_using=None)

G_Paclitaxel = G.subgraph(Paclitaxel_g.nodes())

# add edges between drug and drug target 
G_Paclitaxel.add_edges_from([("Paclitaxel", x) for x in B.neighbors("Paclitaxel")], edge_type = 'drug_targ')

# mutated genes in the subnetwork
common_genes13 = set(G_Paclitaxel.nodes()).intersection(pi_taxel_mutated["Gene name"])
common_pi_taxel_mutated = pi_taxel_mutated[pi_taxel_mutated["Gene name"].isin(common_genes13)]

# add edges between patients and mutated genes 

mutated_pi13 = common_pi_taxel_mutated["icgc_donor_id"].tolist()
mutated_gene13 = common_pi_taxel_mutated["Gene name"].tolist()
G_Paclitaxel.add_edges_from(zip(mutated_pi13, mutated_gene13), edge_type = 'pi_taxel_mu')

nx.write_graphml(G_Paclitaxel, 'Taxel_pi_mu_all.graphml')

nx.info(G_Paclitaxel)

'Name: \nType: Graph\nNumber of nodes: 70\nNumber of edges: 206\nAverage degree:   5.8857'

In [80]:
taxel_mu_genes = pd.DataFrame()
for genes in pi_taxel_mutated['Gene name']:
    taxel_mu_genes[genes] = pd.Series('taxel_mu_genes')

taxel_mu_genes =taxel_mu_genes.T
taxel_mu_genes.to_csv("taxel_mu_genes.noa", sep='\t')

In [81]:
pi_bevac_mutated = drug_mutation1[drug_mutation1['Drug_list'] == 'Bevacizumab']

Bevacizumab_g = nx.from_dict_of_lists(Bevacizumab,create_using=None)

G_Bevacizumab = G.subgraph(Bevacizumab_g.nodes())

# add edges between drug and drug target 
G_Bevacizumab.add_edges_from([("Bevacizumab", x) for x in B.neighbors("Bevacizumab")], edge_type = 'drug_targ')

# mutated genes in the subnetwork
common_genes14 = set(G_Bevacizumab.nodes()).intersection(pi_bevac_mutated["Gene name"])
common_pi_bevac_mutated = pi_bevac_mutated[pi_bevac_mutated["Gene name"].isin(common_genes14)]

# add edges between patients and mutated genes 

mutated_pi14 = common_pi_bevac_mutated["icgc_donor_id"].tolist()
mutated_gene14 = common_pi_bevac_mutated["Gene name"].tolist()
G_Bevacizumab.add_edges_from(zip(mutated_pi14, mutated_gene14), edge_type ='pi_bevac_mu')

nx.write_graphml(G_Bevacizumab, 'Bevac_pi_mu_all.graphml')

nx.info(G_Bevacizumab)

'Name: \nType: Graph\nNumber of nodes: 193\nNumber of edges: 2806\nAverage degree:  29.0777'

In [82]:
bevac_mu_genes = pd.DataFrame()
for genes in pi_bevac_mutated['Gene name']:
    bevac_mu_genes[genes] = pd.Series('bevac_mu_genes')

bevac_mu_genes =bevac_mu_genes.T
bevac_mu_genes.to_csv("bevac_mu_genes.noa", sep='\t')

In [83]:
pi_tras_mutated = drug_mutation1[drug_mutation1['Drug_list'] == 'Trastuzumab']

Trastuzumab_g = nx.from_dict_of_lists(Trastuzumab,create_using=None)

G_Trastuzumab = G.subgraph(Trastuzumab_g.nodes())

# add edges between drug and drug target 
G_Trastuzumab.add_edges_from([("Trastuzumab", x) for x in B.neighbors("Trastuzumab")], edge_type = 'drug_targ')

# mutated genes in the subnetwork
common_genes15 = set(G_Trastuzumab.nodes()).intersection(pi_tras_mutated["Gene name"])
common_pi_tras_mutated = pi_tras_mutated[pi_tras_mutated["Gene name"].isin(common_genes15)]

# add edges between patients and mutated genes 

mutated_pi15 = common_pi_tras_mutated["icgc_donor_id"].tolist()
mutated_gene15 = common_pi_tras_mutated["Gene name"].tolist()
G_Trastuzumab.add_edges_from(zip(mutated_pi15, mutated_gene15), edge_type = 'pi_tras_mu')

nx.write_graphml(G_Trastuzumab, 'Tras_pi_mu_all.graphml')

nx.info(G_Trastuzumab)

'Name: \nType: Graph\nNumber of nodes: 313\nNumber of edges: 4028\nAverage degree:  25.7380'

In [84]:
tras_mu_genes = pd.DataFrame()
for genes in pi_tras_mutated['Gene name']:
    tras_mu_genes[genes] = pd.Series('tras_mu_genes')

tras_mu_genes =tras_mu_genes.T
tras_mu_genes.to_csv("tras_mu_genes.noa", sep='\t')

In [85]:
pi_epi_mutated = drug_mutation1[drug_mutation1['Drug_list'] == 'Epirubicin']

Epirubicin_g = nx.from_dict_of_lists(Epirubicin,create_using=None)

G_Epirubicin = G.subgraph(Epirubicin_g.nodes())

# add edges between drug and drug target 
G_Epirubicin.add_edges_from([("Epirubicin", x) for x in B.neighbors("Epirubicin")], edge_type = 'drug_targ')

# mutated genes in the subnetwork
common_genes16 = set(G_Epirubicin.nodes()).intersection(pi_epi_mutated["Gene name"])
common_pi_epi_mutated = pi_epi_mutated[pi_epi_mutated["Gene name"].isin(common_genes16)]

# add edges between patients and mutated genes 

mutated_pi16 = common_pi_epi_mutated["icgc_donor_id"].tolist()
mutated_gene16 = common_pi_epi_mutated["Gene name"].tolist()
G_Epirubicin.add_edges_from(zip(mutated_pi16, mutated_gene16), edge_type= 'pi_epi_mu')

nx.write_graphml(G_Epirubicin, 'Epi_pi_mu_all.graphml')

nx.info(G_Epirubicin)

'Name: \nType: Graph\nNumber of nodes: 50\nNumber of edges: 224\nAverage degree:   8.9600'

In [86]:
epi_mu_genes = pd.DataFrame()
for genes in pi_epi_mutated['Gene name']:
    epi_mu_genes[genes] = pd.Series('epi_mu_genes')

epi_mu_genes =epi_mu_genes.T
epi_mu_genes.to_csv("epi_mu_genes.noa", sep='\t')

In [87]:
pi_gem_mutated = drug_mutation1[drug_mutation1['Drug_list'] == 'Gemcitabine']

Gemcitabine_g = nx.from_dict_of_lists(Gemcitabine,create_using=None)

G_Gemcitabine = G.subgraph(Gemcitabine_g.nodes())

# add edges between drug and drug target 
G_Gemcitabine.add_edges_from([("Gemcitabine", x) for x in B.neighbors("Gemcitabine")], edge_type = 'drug_targ')

# mutated genes in the subnetwork
common_genes17 = set(G_Gemcitabine.nodes()).intersection(pi_gem_mutated["Gene name"])
common_pi_gem_mutated = pi_gem_mutated[pi_gem_mutated["Gene name"].isin(common_genes17)]

# add edges between patients and mutated genes 

mutated_pi17 = common_pi_gem_mutated["icgc_donor_id"].tolist()
mutated_gene17 = common_pi_gem_mutated["Gene name"].tolist()
G_Gemcitabine.add_edges_from(zip(mutated_pi17, mutated_gene17), edge_type = 'pi_gem_mu')

nx.write_graphml(G_Gemcitabine, 'Gem_pi_mu_all.graphml')

nx.info(G_Gemcitabine)

'Name: \nType: Graph\nNumber of nodes: 78\nNumber of edges: 596\nAverage degree:  15.2821'

In [88]:
gem_mu_genes = pd.DataFrame()
for genes in pi_gem_mutated['Gene name']:
    gem_mu_genes[genes] = pd.Series('gem_mu_genes')

gem_mu_genes =gem_mu_genes.T
gem_mu_genes.to_csv("gem_mu_genes.noa", sep='\t')

In [89]:
pi_alen_mutated = drug_mutation1[drug_mutation1['Drug_list'] == 'Alendronic acid']

Alendronic_acid_g = nx.from_dict_of_lists(Alendronic_acid,create_using=None)

G_Alendronic_acid = G.subgraph(Alendronic_acid_g.nodes())

# add edges between drug and drug target 
G_Alendronic_acid.add_edges_from([("Alendronic acid", x) for x in B.neighbors("Alendronic acid")],edge_type = 'drug_targ')

# mutated genes in the subnetwork
common_genes18 = set(G_Alendronic_acid.nodes()).intersection(pi_alen_mutated["Gene name"])
common_pi_alen_mutated = pi_alen_mutated[pi_alen_mutated["Gene name"].isin(common_genes18)]

# add edges between patients and mutated genes 

mutated_pi18 = common_pi_alen_mutated["icgc_donor_id"].tolist()
mutated_gene18 = common_pi_alen_mutated["Gene name"].tolist()
G_Alendronic_acid.add_edges_from(zip(mutated_pi18, mutated_gene18), edge_type = 'pi_alen_mu')

nx.write_graphml(G_Alendronic_acid, 'Alen_pi_mu_all.graphml')

nx.info(G_Alendronic_acid)

'Name: \nType: Graph\nNumber of nodes: 52\nNumber of edges: 345\nAverage degree:  13.2692'

In [90]:
alen_mu_genes = pd.DataFrame()
for genes in pi_alen_mutated['Gene name']:
    alen_mu_genes[genes] = pd.Series('alen_mu_genes')

alen_mu_genes =alen_mu_genes.T
alen_mu_genes.to_csv("alen_mu_genes.noa", sep='\t')

In [91]:
pi_bus_mutated = drug_mutation1[drug_mutation1['Drug_list'] == 'Buserelin']

Buserelin_g = nx.from_dict_of_lists(Buserelin,create_using=None)

G_Buserelin = G.subgraph(Buserelin_g.nodes())

# add edges between drug and drug target 
G_Buserelin.add_edges_from([("Buserelin", x) for x in B.neighbors("Buserelin")], edge_type='drug_targ')

# mutated genes in the subnetwork
common_genes19 = set(G_Buserelin.nodes()).intersection(pi_bus_mutated["Gene name"])
common_pi_bus_mutated = pi_bus_mutated[pi_bus_mutated["Gene name"].isin(common_genes19)]

# add edges between patients and mutated genes 

mutated_pi19 = common_pi_bus_mutated["icgc_donor_id"].tolist()
mutated_gene19 = common_pi_bus_mutated["Gene name"].tolist()
G_Buserelin.add_edges_from(zip(mutated_pi19, mutated_gene19), edge_type ='pi_mutation')

nx.write_graphml(G_Buserelin, 'Bus_pi_mu_all.graphml')

nx.info(G_Buserelin)

'Name: \nType: Graph\nNumber of nodes: 68\nNumber of edges: 1791\nAverage degree:  52.6765'

In [92]:
bus_mu_genes = pd.DataFrame()
for genes in pi_bus_mutated['Gene name']:
    bus_mu_genes[genes] = pd.Series('bus_mu_genes')

bus_mu_genes =bus_mu_genes.T
bus_mu_genes.to_csv("bus_mu_genes.noa", sep='\t')

In [95]:
zole = target_paths("Zoledronic acid")

In [96]:
zole = target_paths2("Zoledronic acid")

In [97]:
pi_zole_mutated = drug_mutation1[drug_mutation1['Drug_list'] == 'Zoledronic acid']

Zoledronic_acid_g = nx.from_dict_of_lists(zole,create_using=None)

G_Zoledronic_acid = G.subgraph(Zoledronic_acid_g.nodes())

# add edges between drug and drug target 
G_Zoledronic_acid.add_edges_from([("Zoledronic acid", x) for x in B.neighbors("Zoledronic acid")], edge_type='drug_target')

# mutated genes in the subnetwork
common_genes20 = set(G_Zoledronic_acid.nodes()).intersection(pi_zole_mutated["Gene name"])
common_pi_zole_mutated = pi_zole_mutated[pi_zole_mutated["Gene name"].isin(common_genes20)]

# add edges between patients and mutated genes 

mutated_pi20 = common_pi_zole_mutated["icgc_donor_id"].tolist()
mutated_gene20 = common_pi_zole_mutated["Gene name"].tolist()
G_Zoledronic_acid.add_edges_from(zip(mutated_pi20, mutated_gene20), edge_type ='pi_mutation')

nx.write_graphml(G_Zoledronic_acid, 'Zole_pi_mu_all.graphml')

#no common genes between the G_zolendronic_acid and mutated genes 
#given together with Tamoxifen
#if the nbr is increased to even 2nd hop, there's over 1800 nodes and 25000 edges.  

nx.info(G_Zoledronic_acid)

'Name: \nType: Graph\nNumber of nodes: 1867\nNumber of edges: 23055\nAverage degree:  24.6974'

In [97]:
B.neighbors("Zoledronic acid")

['FDPS', 'GGPS1']

In [98]:
C = nx.Graph()               #patients and their mutation graph

C.add_nodes_from(drug_mutation1['icgc_donor_id'], node_type = 'P')

C.add_nodes_from(drug_mutation1['Gene name'], node_type = 'mutated genes')

C.add_edges_from([(row['icgc_donor_id'], row['Gene name']) for idx, row in drug_mutation1.iterrows()], edge_type = 'D_targ')

In [99]:
nx.info(C)

'Name: \nType: Graph\nNumber of nodes: 44392\nNumber of edges: 244251\nAverage degree:  11.0043'

In [ ]:
pi, m = bipartite.sets(C)

In [ ]:
pi_degree = C.degree(pi)

In [ ]:
m_degree = C.degree(m)

In [ ]:
E = nx.Graph()              #patients and the drugs they take graph

E.add_nodes_from(drug_mutation1['icgc_donor_id'], node_type = 'P')

E.add_nodes_from(drug_mutation1['Drug_list'], node_type = 'D')

E.add_edges_from([(row['icgc_donor_id'], row['Drug_list']) for idx, row in drug_mutation1.iterrows()], edge_type = 'pi_drug')

nx.info(E)

In [100]:
len(C.neighbors('DO217786'))       #stable

1761

In [ ]:
len(C.neighbors('DO218168'))         #prog

In [197]:
len(C.neighbors('DO224996'))         #disease progression

1367

In [130]:
drug_list = pd.read_csv("sorted_druglist1.csv")

In [131]:
prog_anas = drug_list[drug_list['Drug_list'] == 'Anastrozole'] 

In [243]:

anas_res = prog_anas.loc[prog_anas['second_therapy_response'].isin(['stable disease','disease progression'])]

In [246]:
anas_mu_res = anas_res.merge(drug_mutation1, how="left")

In [132]:
anas_stable = prog_anas.loc[prog_anas['second_therapy_response'].isin(['stable disease'])]

In [135]:
anas_mu_stbl = anas_stable.merge(drug_mutation1, how="left")

In [248]:
len(set(anas_mu_res['icgc_donor_id']))

3

In [258]:
def target_paths3(node):
    
    drug_to_gene_nodes = B.neighbors(node)
    neighbors = {}
    for dg_node in drug_to_gene_nodes:
        neighbors[dg_node] = {}
        
        if G.has_node(dg_node):
            
            path = nx.single_source_shortest_path(G, dg_node, 3)
            nbr_path = (path)
            neighbors.update(nbr_path)
            
    return(neighbors)

In [137]:
anastrozole = target_paths2("Anastrozole")
anas_res = nx.from_dict_of_lists(anastrozole,create_using=None)

Anas_stable = G.subgraph(anas_res.nodes())

# add edges between drug and drug target 
Anas_stable.add_edges_from([("Anastrozole", x) for x in B.neighbors("Anastrozole")], edge_type = 'drug_targ')

# mutated genes in the subnetwork
common_g1 = set(Anas_stable.nodes()).intersection(anas_mu_stbl["Gene name"])
common_pi_anas_mu1 = anas_mu_stbl[anas_mu_stbl["Gene name"].isin(common_g1)]

# add edges between patients and mutated genes 
mu_pi1 = common_pi_anas_mu1["icgc_donor_id"].tolist()
mutated_g1 = common_pi_anas_mu1["Gene name"].tolist()
Anas_stable.add_edges_from(zip(mu_pi1, mutated_g1), edge_type = 'anas_stab')


nx.write_graphml(Anas_stable, 'Anas_pi_stable.graphml')

nx.info(Anas_stable)

'Name: \nType: Graph\nNumber of nodes: 347\nNumber of edges: 4132\nAverage degree:  23.8156'

In [138]:
len(mutated_g1)

24

In [264]:
anas_mu_g = pd.DataFrame()
for genes in anas_mu_res['Gene name']:
    anas_mu_g[genes] = pd.Series('anas_res')

anas_mu_g =anas_mu_g.T

anas_mu_g.to_csv("anas_res.noa", sep='\t')